In [ ]:
!pip freeze | grep scikit-learn

Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!python -V

Python 3.12.4


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

c:\Users\STUDSENT\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\STUDSENT\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [ ]:
# ======================== Question 1 ========================

# standard devation of the predictions
print('Standard deviation of predictions:', y_pred.std())

Standard deviation of predictions: 6.247488852238703


In [13]:
# ========================= Question 2 ========================

# setting the year and month for the ride_id
year = 2023
month = 3

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

# creating the results dataframe with only ride_id and prediction
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

output_file = "predictions.parquet"

# saving to Parquet
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

df_result.head(10)

,ride_id,predicted_duration
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
5,2023/03_5,10.597174
6,2023/03_6,12.444793
7,2023/03_7,10.972209
8,2023/03_8,23.120343
9,2023/03_9,10.281116


In [ ]:
import os

# getting the size in bytes
file_size_bytes = os.path.getsize(output_file)

# converting to megabytes 
file_size_mb = file_size_bytes / (1024 * 1024)

print(f"Output file size: {file_size_mb:.2f} MB")

Output file size: 65.46 MB
